En este apartado de busca consolidar la bases ya procesadas de forma independiente en una tabla, para las tablas como precipitaciones, temperatura minima se pretende agrupar los datos a nivel diario ya que estos datos tiene mas de un registro por dia detal forma que permita establcer una relacion de muchos a uno con la base agricola

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns   

In [2]:
import os

In [3]:
# Listar los archivos en la ruta actual
archivos_en_ruta_actual = os.listdir('../Datos/Procesados')
print(archivos_en_ruta_actual)

['Base_Agricola.csv', 'Country.csv', 'Fertilizantes.csv', 'Precipitacion.csv', 'Temp_min.csv']


In [4]:
path_temp = '../Datos/Procesados/Temp_min.csv'
path_velocidad = '../Datos/Procesados/Precipitacion.csv'
path_base = '../Datos/Modelado/Base_Agricola_Limpia.csv' 

In [5]:
df_base = pd.read_csv(path_base, sep=';')
df_temp = pd.read_csv(path_temp, sep=';')
df_precip = pd.read_csv(path_velocidad, sep=';')

In [6]:
df_temp.head()

,Fecha,Hora,ValorObservado,NombreEstacion,Municipio,ZonaHidrografica
0,2019-03-22,16:00:00,16.0,SOTAQUIRÁ - AUT,SOTAQUIRÁ,SOGAMOSO
1,2019-05-16,21:58:00,22.8,SAN PABLO DE BORBUR - AUT,SAN PABLO DE BORBUR,MEDIO MAGDALENA
2,2019-06-21,21:00:00,14.1,SOCHA - AUT,SOCHA,SOGAMOSO
3,2019-06-10,20:00:00,15.0,SOGAMOSO - AUT,SOGAMOSO,SOGAMOSO
4,2019-05-08,23:57:00,19.2,SANTA MARIA - AUT,SANTA MARÍA,META


In [7]:
df_precip.head()

,FechaObservacion,ValorObservado,NombreEstacion,Municipio,ZonaHidrografica
0,2019-03-21 19:10:00,0.0,PAUNA - AUT,PAUNA,MEDIO MAGDALENA
1,2019-03-08 15:50:00,0.0,TEATINOS - ESTACION EN PRUEBA,BOYACÁ,META
2,2019-02-26 13:10:00,0.0,BERTHA - AUT,MONIQUIRÁ,SOGAMOSO
3,2019-03-22 07:10:00,0.0,CHITA - AUT,CHITA,SOGAMOSO
4,2019-08-10 18:20:00,0.0,PANELAS - AUT,MOTAVITA,SOGAMOSO


In [8]:
df_precip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11120256 entries, 0 to 11120255
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   FechaObservacion  object 
 1   ValorObservado    float64
 2   NombreEstacion    object 
 3   Municipio         object 
 4   ZonaHidrografica  object 
dtypes: float64(1), object(4)
memory usage: 424.2+ MB


In [9]:
# Agrupar temperatura mínima por día y municipio (promedio)
df_temp_group = df_temp.groupby(['Fecha', 'Municipio'])['ValorObservado'].mean().reset_index()
df_temp_group.rename(columns={'ValorObservado': 'Temp_Min_Promedio'}, inplace=True)
df_temp_group.head()

,Fecha,Municipio,Temp_Min_Promedio
0,2019-01-01,AQUITANIA,8.739706
1,2019-01-01,CHINAVITA,15.046667
2,2019-01-01,CUÍTIVA,8.500000
3,2019-01-01,EL ESPINO,10.220833
4,2019-01-01,GUICÁN,6.623913


In [10]:
df_temp_group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37603 entries, 0 to 37602
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Fecha              37603 non-null  object 
 1   Municipio          37603 non-null  object 
 2   Temp_Min_Promedio  37603 non-null  float64
dtypes: float64(1), object(2)
memory usage: 881.4+ KB


In [11]:

# Agrupar precipitaciones por día y municipio (suma)
df_precip['Fecha'] = pd.to_datetime(df_precip['FechaObservacion'], format='%Y-%m-%d').dt.date.astype(str)
df_precip_group = df_precip.groupby(['Fecha', 'Municipio'])['ValorObservado'].sum().reset_index()
df_precip_group.rename(columns={'ValorObservado': 'Precipitacion_Total'}, inplace=True)
df_precip_group.head()


,Fecha,Municipio,Precipitacion_Total
0,2019-01-01,AQUITANIA,0.0
1,2019-01-01,ARCABUCO,0.0
2,2019-01-01,BOYACÁ,0.0
3,2019-01-01,CHINAVITA,0.0
4,2019-01-01,CHIQUINQUIRÁ,0.0


In [12]:
# Unir las tablas agrupadas con la base agrícola
df_consolidado = df_precip_group.merge(df_temp_group, on=['Fecha', 'Municipio'], how='inner')
# Mostrar las primeras filas del consolidado
df_consolidado.tail(10).sort_values(by='Precipitacion_Total')

,Fecha,Municipio,Precipitacion_Total,Temp_Min_Promedio
35366,2025-04-02,PAIPA,0.0,14.841667
35367,2025-04-02,PAJARITO,0.0,17.343333
35369,2025-04-02,RAMIRIQUÍ,0.0,16.320833
35370,2025-04-02,SAMACÁ,0.0,14.004167
35371,2025-04-02,SOCHA,0.0,14.837500
35374,2025-04-02,TUNJA,0.0,13.295833
35375,2025-04-02,VILLA DE LEIVA,0.0,16.862500
35373,2025-04-02,SOGAMOSO,2.3,14.393478
35372,2025-04-02,SOCOTÁ,7.0,5.922727
35368,2025-04-02,PISVA,1542.2,17.313636


In [14]:
df_consolidado.describe()

,Precipitacion_Total,Temp_Min_Promedio
count,35376.000000,35376.000000
mean,10.239397,13.724264
std,57.649426,4.652722
min,0.000000,-4.700000
25%,0.000000,10.530490
50%,0.500000,13.129167
75%,6.000000,16.945960
max,1809.600000,29.914000


In [15]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11511 entries, 0 to 11510
Data columns (total 29 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   Municipio                                                                 11511 non-null  object 
 1   desagregacionCultivo                                                      11511 non-null  object 
 2   Cultivo                                                                   11511 non-null  object 
 3   Subgrupo                                                                  11511 non-null  object 
 4   Anio                                                                      11511 non-null  int64  
 5   Periodo                                                                   11511 non-null  object 
 6   areaSembradaHa                                                

In [16]:
df_base.head().T

,0,1,2,3,4
Municipio,Tunja,Tunja,Tunja,Tunja,Tunja
desagregacionCultivo,Arracacha,Arracacha,Arracacha,Arracacha,Arveja
Cultivo,Arracacha,Arracacha,Arracacha,Arracacha,Arveja
Subgrupo,Raíces y tubérculos,Raíces y tubérculos,Raíces y tubérculos,Raíces y tubérculos,Leguminosas
Anio,2019,2020,2021,2023,2019
Periodo,A,A,A,B,A
areaSembradaHa,170.0,150.0,180.0,4.0,80.0
areaCosechadaHa,170.0,150.0,180.0,0.0,80.0
produccionTon,2550.0,3750.0,4500.0,0.0,96.0
rendimientoTonHa,15.0,25.0,25.0,0.0,1.2


In [20]:
df_base['Municipio'].unique()

array(['TUNJA', 'ALMEIDA', 'AQUITANIA', 'ARCABUCO', 'BELÉN', 'BERBEO',
       'BETÉITIVA', 'BOAVITA', 'BOYACÁ', 'BRICEÑO', 'BUENAVISTA',
       'BUSBANZÁ', 'CALDAS', 'CAMPOHERMOSO', 'CERINZA', 'CHINAVITA',
       'CHIQUINQUIRÁ', 'CHISCAS', 'CHITA', 'CHITARAQUE', 'CHIVATÁ',
       'CIÉNEGA', 'CÓMBITA', 'COPER', 'CORRALES', 'COVARACHÍA', 'CUBARÁ',
       'CUCAITA', 'CUÍTIVA', 'CHÍQUIZA', 'CHIVOR', 'DUITAMA', 'EL COCUY',
       'EL ESPINO', 'FIRAVITOBA', 'FLORESTA', 'GACHANTIVÁ', 'GÁMEZA',
       'GARAGOA', 'GUACAMAYAS', 'GUATEQUE', 'GUAYATÁ',
       'GÜICÁN DE LA SIERRA', 'IZA', 'JENESANO', 'JERICÓ',
       'LABRANZAGRANDE', 'LA CAPILLA', 'LA VICTORIA', 'LA UVITA',
       'VILLA DE LEYVA', 'MACANAL', 'MARIPÍ', 'MIRAFLORES', 'MONGUA',
       'MONGUÍ', 'MONIQUIRÁ', 'MOTAVITA', 'MUZO', 'NOBSA', 'NUEVO COLÓN',
       'OICATÁ', 'OTANCHE', 'PACHAVITA', 'PÁEZ', 'PAIPA', 'PAJARITO',
       'PANQUEBA', 'PAUNA', 'PAYA', 'PAZ DE RÍO', 'PESCA', 'PISBA',
       'PUERTO BOYACÁ', 'QUÍPAMA', 'RAMIRIQUÍ'

In [21]:
# Convertir los nombres de los municipios a mayúsculas
df_base['Municipio'] = df_base['Municipio'].str.upper()

# Verificar los valores únicos en la columna 'Municipio'
print(df_base['Municipio'].unique())

['TUNJA' 'ALMEIDA' 'AQUITANIA' 'ARCABUCO' 'BELÉN' 'BERBEO' 'BETÉITIVA'
 'BOAVITA' 'BOYACÁ' 'BRICEÑO' 'BUENAVISTA' 'BUSBANZÁ' 'CALDAS'
 'CAMPOHERMOSO' 'CERINZA' 'CHINAVITA' 'CHIQUINQUIRÁ' 'CHISCAS' 'CHITA'
 'CHITARAQUE' 'CHIVATÁ' 'CIÉNEGA' 'CÓMBITA' 'COPER' 'CORRALES'
 'COVARACHÍA' 'CUBARÁ' 'CUCAITA' 'CUÍTIVA' 'CHÍQUIZA' 'CHIVOR' 'DUITAMA'
 'EL COCUY' 'EL ESPINO' 'FIRAVITOBA' 'FLORESTA' 'GACHANTIVÁ' 'GÁMEZA'
 'GARAGOA' 'GUACAMAYAS' 'GUATEQUE' 'GUAYATÁ' 'GÜICÁN DE LA SIERRA' 'IZA'
 'JENESANO' 'JERICÓ' 'LABRANZAGRANDE' 'LA CAPILLA' 'LA VICTORIA'
 'LA UVITA' 'VILLA DE LEYVA' 'MACANAL' 'MARIPÍ' 'MIRAFLORES' 'MONGUA'
 'MONGUÍ' 'MONIQUIRÁ' 'MOTAVITA' 'MUZO' 'NOBSA' 'NUEVO COLÓN' 'OICATÁ'
 'OTANCHE' 'PACHAVITA' 'PÁEZ' 'PAIPA' 'PAJARITO' 'PANQUEBA' 'PAUNA' 'PAYA'
 'PAZ DE RÍO' 'PESCA' 'PISBA' 'PUERTO BOYACÁ' 'QUÍPAMA' 'RAMIRIQUÍ'
 'RÁQUIRA' 'RONDÓN' 'SABOYÁ' 'SÁCHICA' 'SAMACÁ' 'SAN EDUARDO'
 'SAN JOSÉ DE PARE' 'SAN LUIS DE GACENO' 'SAN MATEO' 'SAN MIGUEL DE SEMA'
 'SAN PABLO DE BORBUR' 'SANTANA

In [22]:
df_consolidado.Municipio.unique()

array(['AQUITANIA', 'CHINAVITA', 'CUÍTIVA', 'EL ESPINO', 'LA CAPILLA',
       'PAIPA', 'PISVA', 'SAN PABLO DE BORBUR', 'SANTA MARÍA', 'SOGAMOSO',
       'VENTAQUEMADA', 'ZETAQUIRÁ', 'CHITA', 'GUAYATÁ', 'MONGUA',
       'MOTAVITA', 'PESCA', 'QUÍPAMA', 'SAMACÁ', 'SIACHOQUE', 'SOTAQUIRÁ',
       'VILLA DE LEIVA', 'SOCOTÁ', 'SOCHA', 'BOYACÁ', 'GUACAMAYAS',
       'PÁEZ', 'PUERTO BOYACÁ', 'PAJARITO', 'MACANAL', 'TUNJA',
       'BUENAVISTA', 'CORRALES', 'OTANCHE', 'RAMIRIQUÍ'], dtype=object)

In [23]:
# Crear la columna 'Periodo' basada en la lógica de los meses
df_consolidado['Periodo'] = df_consolidado['Fecha'].apply(
    lambda x: 'A' if int(x.split('-')[1]) <= 6 else 'B'
)

# Verificar el resultado
print(df_consolidado[['Fecha', 'Periodo']].tail())

            Fecha Periodo
35371  2025-04-02       A
35372  2025-04-02       A
35373  2025-04-02       A
35374  2025-04-02       A
35375  2025-04-02       A


In [25]:
df_consolidado.head()

,Fecha,Municipio,Precipitacion_Total,Temp_Min_Promedio,Periodo
0,2019-01-01,AQUITANIA,0.0,8.739706,A
1,2019-01-01,CHINAVITA,0.0,15.046667,A
2,2019-01-01,CUÍTIVA,0.0,8.500000,A
3,2019-01-01,EL ESPINO,0.0,10.220833,A
4,2019-01-01,LA CAPILLA,0.0,17.357143,A


In [32]:
df_base[(df_base['Municipio'] == 'CHINAVITA') & (df_base['Anio'] == 2020)].head()

,Municipio,desagregacionCultivo,Cultivo,Subgrupo,Anio,Periodo,areaSembradaHa,areaCosechadaHa,produccionTon,rendimientoTonHa,...,GpOleaginosas,grupoCultivo_Raíces y tubérculos,Boyaca,EdoCañaVerde,Edofresco,EdoFibraCabuya,EdoGrano,EdoGranoSeco,EdoPaddyCascaraVerde,EdoSecoTrilla
1262,CHINAVITA,Achira,Achira,Raíces y tubérculos,2020,A,11.0,11.0,44.0,4.0,...,0,1,1,0,1,0,0,0,0,0
1263,CHINAVITA,Achira,Achira,Raíces y tubérculos,2020,B,2.0,2.0,12.0,6.0,...,0,1,1,0,1,0,0,0,0,0
1269,CHINAVITA,Aguacate demás variedades,Aguacate,Demás frutales,2020,C,28.0,28.0,140.0,5.0,...,0,0,1,0,1,0,0,0,0,0
1275,CHINAVITA,Arracacha,Arracacha,Raíces y tubérculos,2020,A,2.5,2.5,25.0,10.0,...,0,1,1,0,1,0,0,0,0,0
1276,CHINAVITA,Arracacha,Arracacha,Raíces y tubérculos,2020,B,1.0,1.0,10.0,10.0,...,0,1,1,0,1,0,0,0,0,0


In [35]:
# Agregar columna de año en df_consolidado
df_consolidado['Anio'] = pd.to_datetime(df_consolidado['Fecha']).dt.year

df_consolidado_anual_C = df_consolidado.groupby(['Anio', 'Municipio']).agg({
    'Precipitacion_Total': 'sum',
    'Temp_Min_Promedio': 'mean'
}).reset_index()
df_consolidado_anual_C['Periodo'] = 'C'

# Agrupar por año y municipio, calculando la suma de precipitaciones y el promedio de temperatura mínima
df_consolidado_anual = df_consolidado.groupby(['Anio', 'Municipio','Periodo']).agg({
    'Precipitacion_Total': 'sum',
    'Temp_Min_Promedio': 'mean'
}).reset_index()

# Unir los registros de periodo A, B y C en un solo DataFrame
df_consolidado_anual_completo = pd.concat([df_consolidado_anual, df_consolidado_anual_C], ignore_index=True)

# Mostrar el resultado


# Unir con df_base usando año y municipio
df_base_consolidado = df_base.merge(
    df_consolidado_anual_completo,
    left_on=['Anio', 'Municipio','Periodo'],
    right_on=['Anio', 'Municipio', 'Periodo'],
    how='left'
)

In [41]:
df_consolidado_anual_completo[df_consolidado_anual_completo['Municipio'] == 'TUNJA'].head()

,Anio,Municipio,Periodo,Precipitacion_Total,Temp_Min_Promedio
262,2024,TUNJA,B,549.6,12.721638
286,2025,TUNJA,A,72.2,12.768576
442,2024,TUNJA,C,549.6,12.721638
466,2025,TUNJA,C,72.2,12.768576


In [42]:
df_base_consolidado[(df_base_consolidado['Municipio'] == 'TUNJA') & (df_base_consolidado['Anio'] == 2020)].head()

,Municipio,desagregacionCultivo,Cultivo,Subgrupo,Anio,Periodo,areaSembradaHa,areaCosechadaHa,produccionTon,rendimientoTonHa,...,Boyaca,EdoCañaVerde,Edofresco,EdoFibraCabuya,EdoGrano,EdoGranoSeco,EdoPaddyCascaraVerde,EdoSecoTrilla,Precipitacion_Total,Temp_Min_Promedio
1,TUNJA,Arracacha,Arracacha,Raíces y tubérculos,2020,A,150.0,150.0,3750.000000,25.0,...,1,0,1,0,0,0,0,0,NaN,NaN
6,TUNJA,Arveja,Arveja,Leguminosas,2020,A,20.0,20.0,56.074766,2.8,...,1,0,0,0,1,0,0,0,NaN,NaN
7,TUNJA,Arveja,Arveja,Leguminosas,2020,B,200.0,200.0,560.747664,2.8,...,1,0,0,0,1,0,0,0,NaN,NaN
16,TUNJA,Avena,Avena,Cereales,2020,A,160.0,160.0,480.000000,3.0,...,1,0,1,0,0,0,0,0,NaN,NaN
17,TUNJA,Avena,Avena,Cereales,2020,B,100.0,100.0,300.000000,3.0,...,1,0,1,0,0,0,0,0,NaN,NaN


In [38]:
df_base_consolidado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11511 entries, 0 to 11510
Data columns (total 31 columns):
 #   Column                                                                    Non-Null Count  Dtype  
---  ------                                                                    --------------  -----  
 0   Municipio                                                                 11511 non-null  object 
 1   desagregacionCultivo                                                      11511 non-null  object 
 2   Cultivo                                                                   11511 non-null  object 
 3   Subgrupo                                                                  11511 non-null  object 
 4   Anio                                                                      11511 non-null  int64  
 5   Periodo                                                                   11511 non-null  object 
 6   areaSembradaHa                                                

In [39]:
df_base_consolidado[df_base_consolidado['Precipitacion_Total'].isna()]['Municipio'].unique()

array(['TUNJA', 'ALMEIDA', 'ARCABUCO', 'BELÉN', 'BERBEO', 'BETÉITIVA',
       'BOAVITA', 'BOYACÁ', 'BRICEÑO', 'BUENAVISTA', 'BUSBANZÁ', 'CALDAS',
       'CAMPOHERMOSO', 'CERINZA', 'CHINAVITA', 'CHIQUINQUIRÁ', 'CHISCAS',
       'CHITA', 'CHITARAQUE', 'CHIVATÁ', 'CIÉNEGA', 'CÓMBITA', 'COPER',
       'CORRALES', 'COVARACHÍA', 'CUBARÁ', 'CUCAITA', 'CHÍQUIZA',
       'CHIVOR', 'DUITAMA', 'EL COCUY', 'EL ESPINO', 'FIRAVITOBA',
       'FLORESTA', 'GACHANTIVÁ', 'GÁMEZA', 'GARAGOA', 'GUACAMAYAS',
       'GUATEQUE', 'GUAYATÁ', 'GÜICÁN DE LA SIERRA', 'IZA', 'JENESANO',
       'JERICÓ', 'LABRANZAGRANDE', 'LA CAPILLA', 'LA VICTORIA',
       'LA UVITA', 'VILLA DE LEYVA', 'MACANAL', 'MARIPÍ', 'MIRAFLORES',
       'MONGUA', 'MONGUÍ', 'MONIQUIRÁ', 'MOTAVITA', 'MUZO', 'NOBSA',
       'NUEVO COLÓN', 'OICATÁ', 'OTANCHE', 'PACHAVITA', 'PÁEZ', 'PAIPA',
       'PAJARITO', 'PANQUEBA', 'PAUNA', 'PAYA', 'PAZ DE RÍO', 'PESCA',
       'PISBA', 'PUERTO BOYACÁ', 'QUÍPAMA', 'RAMIRIQUÍ', 'RÁQUIRA',
       'RONDÓN', '